In [2]:
import os 
#os.chdir('../')
os.getcwd()

'C:\\Users\\Jiajie Zhang\\flowers102'

In [124]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.layers import Flatten, Dense, Dropout, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import math

In [90]:
train_path = 'dataset/flower_data_10/train'
valid_path = 'dataset/flower_data_10/valid'

In [86]:
nb_epoch = 20
batch_size = 32
img_size = (224,224)
classes = list(map(str,[1,2,3,4,5,6,7,8,9,10]))
rgb_mean = [123.68, 116.779, 103.939]

In [ ]:
def fc_block(X,units,dropout,stage):
    fc_name = 'fc' + str(stage)
    X = Dense(units,activation ='elu',name = fc_name)(X)
    X = Dropout(dropout)(X)
    return X

In [120]:
def ResNet50_transfer():
    
    #call base_model
    base_model = ResNet50(
        include_top=False,
        weights="imagenet",
        input_tensor= Input(shape=img_size + (3,))
    )
    
    # freeze resnet layers' params
    for layer in base_model.layers: 
        layer.trainable = False
        
    # top architecture
    X = base_model.output
    X = Flatten()(X)
    X = Dropout(0.4)(X)
    X = fc_block(X,fc_layer_units[0],dropout = 0.4,stage = 1)
    X = fc_block(X,fc_layer_units[1],dropout = 0.4,stage = 2)

    # output layer
    X = Dense(len(classes),activation='softmax',name = 'fc3_output')(X)

    # create model
    model = Model(inputs = base_model.input,outputs = X, name = 'ResNet50_transfer')
    
    return model

In [127]:
def generate_data(train_path,valid_path):
    # generate training data 
    train_datagen = ImageDataGenerator(rotation_range=30., shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    train_datagen.mean = np.array([123.675, 116.28 , 103.53], dtype=np.float32).reshape((3, 1, 1))
    train_data = train_datagen.flow_from_directory(train_path, target_size=img_size, classes=None)
    # generate training data 
    valid_datagen = ImageDataGenerator()
    valid_datagen.mean = np.array([123.675, 116.28 , 103.53], dtype=np.float32).reshape((3, 1, 1))
    valid_data = train_datagen.flow_from_directory(valid_path, target_size=img_size, classes=None)
    return train_data, valid_data

In [130]:
def call_back():
    early_stopping = EarlyStopping(verbose=1, patience=10, monitor='val_loss')  
    model_checkpoint = ModelCheckpoint(filepath='102flowersmodel.h5', verbose=1, save_best_only=True, monitor='val_loss')
    callbacks = [early_stopping, model_checkpoint]
    return callbacks

In [133]:
model = ResNet50_transfer()

In [134]:
model.summary()

Model: "ResNet50_transfer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________

In [135]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

In [136]:
train_data, valid_data = generate_data(train_path,valid_path)

Found 427 images belonging to 10 classes.
Found 43 images belonging to 10 classes.


In [137]:
callbacks = call_back()

In [138]:
model.fit_generator(train_data, steps_per_epoch= math.ceil(train_data.samples / batch_size), epochs=nb_epoch,
                    validation_data=valid_data, validation_steps=math.ceil(valid_data.samples / batch_size),
                    callbacks=callbacks)

Epoch 1/20
14/14 [==============================] - 68s 5s/step - loss: 4.9959 - accuracy: 0.1639 - val_loss: 2.2945 - val_accuracy: 0.4651

Epoch 00001: val_loss improved from inf to 2.29451, saving model to 102flowersmodel.h5
Epoch 2/20
14/14 [==============================] - 49s 4s/step - loss: 3.1324 - accuracy: 0.3419 - val_loss: 1.0720 - val_accuracy: 0.6512

Epoch 00002: val_loss improved from 2.29451 to 1.07198, saving model to 102flowersmodel.h5
Epoch 3/20
14/14 [==============================] - 50s 4s/step - loss: 2.0920 - accuracy: 0.4660 - val_loss: 1.8200 - val_accuracy: 0.5581

Epoch 00003: val_loss did not improve from 1.07198
Epoch 4/20
14/14 [==============================] - 57s 4s/step - loss: 1.6612 - accuracy: 0.5386 - val_loss: 1.5891 - val_accuracy: 0.6977

Epoch 00004: val_loss did not improve from 1.07198
Epoch 5/20
14/14 [==============================] - 56s 4s/step - loss: 1.4454 - accuracy: 0.6206 - val_loss: 1.0487 - val_accuracy: 0.6047

Epoch 00005: va